In [1]:
import sqlite3

In [6]:
import sqlite3

path = ''
db_name = 'github_repos.db'   

try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()

    sql = '''
    CREATE TABLE IF NOT EXISTS repos (
        id INTEGER PRIMARY KEY,
        name TEXT,
        language TEXT,
        stars INTEGER
    );
    '''

    cur.execute(sql)
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    conn.close()


In [8]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import time

# === GitHubユーザー名（あなた） ===
username = "Yua-Kanno"

# === スクレイピング対象URL ===
base_url = f"https://github.com/{username}?tab=repositories"

# === DB名 ===
db_name = "github_repos.db"

# ==== DB作成 ====
conn = sqlite3.connect(db_name)
cur = conn.cursor()

cur.execute("""
    CREATE TABLE IF NOT EXISTS repos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        language TEXT,
        stars INTEGER
    );
""")

# 前のデータを削除
cur.execute("DELETE FROM repos;")
conn.commit()

# ==== スクレイピング開始 ====

print("スクレイピング中...")

response = requests.get(base_url)
soup = BeautifulSoup(response.text, "html.parser")

# 各リポジトリのHTMLブロックを取得
repo_items = soup.select("li[source='repo']") or soup.select("li.Box-row")

time.sleep(1)  # ← アクセス間隔（課題必須）

for repo in repo_items:
    # --- リポジトリ名 ---
    name_tag = repo.select_one("a[itemprop='name codeRepository']")
    if not name_tag:
        continue

    name = name_tag.text.strip()

    # --- 言語 ---
    lang_tag = repo.select_one("span[itemprop='programmingLanguage']")
    language = lang_tag.text.strip() if lang_tag else "不明"

    # --- スター数 ---
    star_tag = repo.select_one("a[href*='/stargazers']")
    if star_tag:
        try:
            stars = int(star_tag.text.strip().replace(",", ""))
        except:
            stars = 0
    else:
        stars = 0

    # === DB保存 ===
    cur.execute(
        "INSERT INTO repos (name, language, stars) VALUES (?, ?, ?)",
        (name, language, stars)
    )
    conn.commit()

print("保存完了！\n")

# ==== SELECTで表示 ====
print("=== 保存されたデータ ===")
cur.execute("SELECT * FROM repos;")
rows = cur.fetchall()

for row in rows:
    print(row)

conn.close()


スクレイピング中...
保存完了！

=== 保存されたデータ ===
